<a href="https://colab.research.google.com/github/adamrhurley/FYP/blob/main/MusicRecommenderSystem/FailedKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Reference: https://www.analyticsvidhya.com/blog/2020/08/recommendation-system-k-nearest-neighbors/

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import base64
import io
from matplotlib.pyplot import imread
import codecs
from IPython.display import HTML

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_w_genres.csv to data_w_genres.csv


In [ ]:
artists = pd.read_csv(io.BytesIO(uploaded['data_w_genres.csv']))
artists["id"] = artists.index + 1

In [ ]:
artists = artists[artists.genres != "[]"]
del artists["acousticness"]
del artists["danceability"]
del artists["duration_ms"]
del artists["energy"]
del artists["instrumentalness"]
del artists["liveness"]
del artists["loudness"]
del artists["speechiness"]
del artists["tempo"]
del artists["valence"]
del artists["popularity"]
del artists["key"]
del artists["mode"]



In [ ]:
artists = artists[['id',"artists","genres","count"]]

In [ ]:
artists.head(100)

,id,artists,genres,count
0,1,"""Cats"" 1981 Original London Cast",['show tunes'],10
8,9,"""Weird Al"" Yankovic","['comedy rock', 'comic', 'parody']",106
10,11,$NOT,"['florida rap', 'underground hip hop', 'vapor ...",6
11,12,$atori Zoom,"['dark trap', 'meme rap']",2
13,14,$uicideBoy$,"['dark trap', 'new orleans rap', 'underground ...",69
...,...,...,...,...
172,173,6LACK,"['alternative r&b', 'atl hip hop', 'hip hop', ...",21
173,174,6ix9ine,['emo rap'],11
174,175,7 Seconds,"['hardcore', 'hardcore punk', 'melodic hardcor...",2
175,176,7 Skies,['uplifting trance'],6


In [ ]:
artists.iloc[50]

id                                                       106
artists                                                2Baba
genres     ['afro dancehall', 'afropop', 'azonto', 'azont...
count                                                      2
Name: 105, dtype: object

In [ ]:
artists['genres'] = artists['genres'].str.strip('[]').str.replace(' ','').str.replace("'",'')
artists['genres'] = artists['genres'].str.split(',')

In [ ]:
for i,j in zip(artists['genres'],artists.index):
    list2=[]
    list2=i
    list2.sort()
    artists.loc[j,'genres']=str(list2)
artists['genres'] = artists['genres'].str.strip('[]').str.replace(' ','').str.replace("'",'')
artists['genres'] = artists['genres'].str.split(',')

In [ ]:
genreList = []
for index, row in artists.iterrows():
    genres = row["genres"]
    
    for genre in genres:
        if genre not in genreList:
            genreList.append(genre)
genreList[:10] #now we have a list with unique genres

['showtunes',
 'comedyrock',
 'comic',
 'parody',
 'floridarap',
 'undergroundhiphop',
 'vaportrap',
 'darktrap',
 'memerap',
 'neworleansrap']

In [ ]:
def binary(genre_list):
    binaryList = []
    
    for genre in genreList:
        if genre in genre_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList

In [ ]:
artists['genres_bin'] = artists['genres'].apply(lambda x: binary(x))
artists['genres_bin'].head()

0     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8     [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
10    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
11    [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...
13    [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...
Name: genres_bin, dtype: object

In [ ]:
from scipy import spatial

def Similarity(artist1, artist2):
    a = artists.iloc[artist1]
    b = artists.iloc[artist2]

    genresA = a['genres_bin']
    genresB = b['genres_bin']
    
    genreDistance = spatial.distance.cosine(genresA, genresB)

    return genreDistance

In [ ]:
Similarity(8,13)

1.0

In [ ]:
import operator

def predict_score(name):
    new_artists = artists[artists['artists'].str.contains(name)].iloc[0].to_frame().T
    print('Selected Movie: ',new_artists.artists.values[0])
    def getNeighbors(baseArtist, K):
        distances = []
    
        for index, artist in artists.iterrows():
            if artist['id'] != baseArtist['id'].values[0]:
                dist = Similarity(baseArtist['id'].values[0], artist['id'])
                distances.append((artists['id'], dist))
    
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
    
        for x in range(K):
            neighbors.append(distances[x])
        return neighbors

    K = 5
    avgRating = 0
    neighbors = getNeighbors(new_artists, K)
    
    print('\nRecommended Artists: \n')
    for neighbor in neighbors:
        avgRating = avgRating+artists.iloc[neighbor[0]][1]  
        print( artists.iloc[neighbor[0]][0]+" | Genres: "+str(artists.iloc[neighbor[0]][1]).strip('[]').replace(' ','')+" | Rating: "+str(artists.iloc[neighbor[0]][1]))
    
    print('\n')
    avgRating = avgRating/K
    print('The predicted rating for %s is: %f' %(new_artists['artists'].values[0],avgRating))
    print('The actual rating for %s is %f' %(new_artists['artists'].values[0],new_artists['count']))

In [ ]:
predict_score('6ix9ine')

Selected Movie:  6ix9ine


IndexError: ignored